# Analyse Globale des Données CAN - CANlock

## Objectif
Ce notebook permet d'obtenir un aperçu global des ~490 millions de messages CAN pour mieux comprendre la structure des données et identifier les patterns normaux.

## Analyses Réalisées
1. Proportion de SPNs analogiques vs catégoriques
2. Proportion du PGN 60416
3. Liste de tous les SPNs et leurs fréquences
4. Messages avec DLC=0
5. Niveaux de priorité des messages
6. Analyses avancées pour la détection d'anomalies

## Configuration
- **Test rapide**: `max_batches=50` → ~5M messages en 5-10 min
- **Analyse complète**: `max_batches=None` → ~490M messages en 4-8h (à lancer la nuit)
- **Cache**: Les résultats sont sauvegardés automatiquement dans `../data/cache/`

---
## Imports et Initialisation

**But**: Charger toutes les librairies nécessaires et initialiser la connexion à la base de données.

In [13]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sqlmodel import select
from collections import defaultdict, Counter
from sqlalchemy import func
from tqdm import tqdm
import pickle
from pathlib import Path

from canlock.db.database import get_session, init_db
from canlock.db.models import (
    SpnDefinition, AnalogAttributes, DefinedDigitalValues, 
    Session, CanMessage, PgnDefinition
)
from canlock.decoder import SessionDecoder

# Initialisation de la base de données
init_db()

# Configuration
BATCH_SIZE = 100000  # Nombre de messages traités à la fois
CACHE_DIR = Path('../data/cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("Imports et initialisation terminés")

Imports et initialisation terminés


---
## Fonction Helper: Mapping PGN -> Nom

In [14]:
def get_pgn_name_mapping():
    """Récupère le mapping PGN ID -> Nom depuis la base de données"""
    with get_session() as session:
        pgn_defs = session.exec(select(PgnDefinition)).all()
        mapping = {}
        for pgn_def in pgn_defs:
            if pgn_def.name:
                mapping[pgn_def.pgn_identifier] = pgn_def.name
            else:
                mapping[pgn_def.pgn_identifier] = f"PGN_{pgn_def.pgn_identifier}"
        return mapping

def get_pgn_display_name(pgn_id, pgn_mapping):
    """Retourne le nom du PGN ou 'PGN_xxxxx' si pas de nom"""
    return pgn_mapping.get(pgn_id, f"PGN_{pgn_id}")

# Charger le mapping global
print("Chargement du mapping PGN -> Nom...")
PGN_NAME_MAPPING = get_pgn_name_mapping()
print(f"Mapping chargé: {len(PGN_NAME_MAPPING)} PGNs définis")

Chargement du mapping PGN -> Nom...
Mapping chargé: 2520 PGNs définis


---
## 1. Analyse des SPNs: Analogiques vs Catégoriques

### But
Déterminer la proportion de SPNs (Suspect Parameter Numbers) qui sont:
- **Analogiques** (`is_analog = True`): valeurs continues (vitesse, température, etc.)
- **Catégoriques** (`is_analog = False`): valeurs discrètes/états (marche/arrêt, modes, etc.)

### Pourquoi c'est important
- Les SPNs analogiques et catégoriques nécessitent des approches différentes pour la détection d'anomalies
- Permet de comprendre la nature des données disponibles

In [15]:
# Récupérer toutes les définitions SPN depuis la BD
with get_session() as session:
    spns = session.exec(select(SpnDefinition)).all()
    total_spns = len(spns)
    analog_count = sum(1 for s in spns if s.is_analog)
    digital_count = total_spns - analog_count

# Affichage des résultats
print(f"RÉSULTATS - Proportions SPNs")
print(f"{'='*50}")
print(f"Total SPNs définis: {total_spns:,}")
print(f"SPNs Analogiques: {analog_count:,} ({100*analog_count/total_spns:.2f}%)")
print(f"SPNs Catégoriques: {digital_count:,} ({100*digital_count/total_spns:.2f}%)")

# Visualisation
df_prop = pd.DataFrame({
    'Type': ['Analogique', 'Catégorique'], 
    'Count': [analog_count, digital_count]
})

fig = px.pie(df_prop, values='Count', names='Type', 
             title='Distribution des SPNs: Analogiques vs Catégoriques',
             color_discrete_sequence=['#3498db', '#e74c3c'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

RÉSULTATS - Proportions SPNs
Total SPNs définis: 13,313
SPNs Analogiques: 13,313 (100.00%)
SPNs Catégoriques: 0 (0.00%)


### Interprétation des Résultats

- Les données sont principalement des mesures continues
- Modèles de régression et détection d'outliers seront pertinents


---
## 2. Liste Complète des SPNs

### But
Obtenir la liste de tous les SPNs disponibles avec leurs caractéristiques:
- ID du SPN
- Nom (ex: ENGINE_SPEED, VEHICLE_SPEED)
- Type (analogique/catégorique)
- Position dans le message (bit_start, bit_length)

### Pourquoi c'est important
- Identifier quels signaux sont disponibles pour l'analyse
- Comprendre la structure des messages CAN

In [16]:
# Créer un DataFrame avec toutes les infos SPN
with get_session() as session:
    spn_defs = session.exec(select(SpnDefinition)).all()
    
df_spns = pd.DataFrame([{
    'spn_id': s.spn_identifier,
    'spn_name': s.name,
    'is_analog': s.is_analog,
    'bit_start': s.bit_start,
    'bit_length': s.bit_length
} for s in spn_defs])

# Statistiques
print(f"RÉSULTATS - Liste des SPNs")
print(f"{'='*50}")
print(f"Total SPNs: {len(df_spns):,}")
print(f"SPNs avec nom défini: {df_spns['spn_name'].notna().sum():,}")
print(f"SPNs sans nom: {df_spns['spn_name'].isna().sum():,}")

# Afficher les SPNs nommés (les plus importants)
print("\nExemples de SPNs nommés (premiers 20):")
display(df_spns[df_spns['spn_name'].notna()].sort_values('spn_name').head(20))

RÉSULTATS - Liste des SPNs
Total SPNs: 13,313
SPNs avec nom défini: 13,313
SPNs sans nom: 0

Exemples de SPNs nommés (premiers 20):


,spn_id,spn_name,is_analog,bit_start,bit_length
9543,6378,2_WHEEL_STEER_ACTUATOR_STATE,True,8,2
9544,6379,4_WHEEL_STEER_ACTUATOR_STATE,True,10,2
12007,1737,ABOVE_NOMINAL_LEVEL_FRONT_AXLE,True,12,2
12008,1736,ABOVE_NOMINAL_LEVEL_REAR_AXLE,True,14,2
9123,13131,ABSOLUTE_ENGINE_LOAD_PERCENT_AIR_MASS,True,40,16
12116,2794,ABSOLUTE_LASER_STRIKE_POSITION,True,40,16
6031,21101,ABSOLUTE_STEERING_POSITION_QUALITY_FACTOR,True,22,2
1062,1438,ABS_EBS_AMBER_WARNING_SIGNAL_POWERED_VEHICLE,True,44,2
1060,1243,ABS_FULLY_OPERATIONAL,True,40,2
1051,575,ABS_OFF_ROAD_SWITCH,True,16,2


### Interprétation

Les SPNs nommés sont généralement les plus importants pour l'analyse car ils correspondent à des paramètres bien documentés du protocole J1939 (ex: vitesse moteur, température, pression).

---
## 3. Fonction d'Analyse par Lots

### But
Parcourir efficacement les ~490M messages CAN pour extraire toutes les statistiques nécessaires.

### Stratégie
- **Batch processing**: Traite 100k messages à la fois pour gérer la mémoire
- **Cache**: Sauvegarde automatique des résultats pour ne pas recalculer
- **Progress tracking**: Barre de progression pour suivre l'avancement

### Données Collectées
- Compteurs de PGNs (Parameter Group Numbers)
- Distribution des DLC (Data Length Code)
- Niveaux de priorité des messages
- CAN IDs uniques

In [1]:
def analyze_can_messages_batch(max_batches=None, use_cache=True):
    """
    Analyse les messages CAN par lots.
    
    Args:
        max_batches: Nombre maximum de lots à traiter (None = tous)
                     Exemple: 50 batches = ~5M messages (~5-10 min)
        use_cache: Si True, charge les résultats du cache si disponibles
    
    Returns:
        dict: Statistiques complètes des messages CAN
    """
    cache_file = CACHE_DIR / 'can_analysis_cache.pkl'
    
    # Charger depuis le cache si disponible
    if use_cache and cache_file.exists():
        print("Chargement des résultats depuis le cache...")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    print("Démarrage de l'analyse...")
    
    # Initialisation des statistiques
    stats = {
        'total_messages': 0,
        'pgn_counts': Counter(),
        'dlc_counts': Counter(),
        'priority_counts': Counter(),
        'can_id_counts': Counter(),
    }
    
    offset = 0
    batch_index = 0
    
    with get_session() as session:
        total_count = session.exec(select(func.count()).select_from(CanMessage)).one()
        print(f"Total messages dans la BD: {total_count:,}")
        
        total_batches = (total_count // BATCH_SIZE) + 1
        if max_batches:
            total_batches = min(total_batches, max_batches)
        
        print(f"Traitement de {total_batches} lots de {BATCH_SIZE:,} messages...")
        
        with tqdm(total=total_batches, desc="Progression") as pbar:
            while True:
                if max_batches and batch_index >= max_batches:
                    break
                
                q = select(CanMessage).offset(offset).limit(BATCH_SIZE)
                batch = session.exec(q).all()
                
                if not batch:
                    break
                
                for msg in batch:
                    stats['total_messages'] += 1
                    
                    if msg.length is not None:
                        stats['dlc_counts'][msg.length] += 1
                    
                    if msg.can_identifier is not None:
                        stats['can_id_counts'][msg.can_identifier] += 1
                        
                        try:
                            pgn = SessionDecoder.extract_pgn_number_from_payload(msg.can_identifier)
                            stats['pgn_counts'][pgn] += 1
                            
                            priority = (msg.can_identifier >> 26) & 0x7
                            stats['priority_counts'][priority] += 1
                        except Exception:
                            pass
                
                offset += BATCH_SIZE
                batch_index += 1
                pbar.update(1)
    
    print("\nSauvegarde des résultats dans le cache...")
    with open(cache_file, 'wb') as f:
        pickle.dump(stats, f)
    
    print("Analyse terminée!")
    return stats

---
## 4. Exécution de l'Analyse

### IMPORTANT - Choisir le Mode

**Option 1 - TEST RAPIDE (RECOMMANDÉ POUR COMMENCER)**:
```python
stats = analyze_can_messages_batch(max_batches=50, use_cache=True)
```
- Analyse ~5M messages
- Temps: 5-10 minutes
- Idéal pour tester et développer

**Option 2 - ANALYSE COMPLÈTE**:
```python
stats = analyze_can_messages_batch(max_batches=None, use_cache=True)
```
- Analyse TOUS les ~490M messages
- Temps: 4-8 heures
- À lancer la nuit ou sur une machine dédiée

### Astuce
Le cache permet de charger instantanément les résultats si l'analyse a déjà été faite. Pour forcer un recalcul: `use_cache=False`

In [ ]:
# OPTION 1: Test rapide (~5M messages, ~5-10 min)
stats = analyze_can_messages_batch(max_batches=50, use_cache=True)

# OPTION 2: Analyse complète (~490M messages, plusieurs heures)
# stats = analyze_can_messages_batch(max_batches=None, use_cache=True)

# Résumé des résultats
print(f"\n{'='*60}")
print(f"STATISTIQUES GLOBALES")
print(f"{'='*60}")
print(f"Messages analysés: {stats['total_messages']:,}")
print(f"PGNs uniques trouvés: {len(stats['pgn_counts']):,}")
print(f"CAN IDs uniques: {len(stats['can_id_counts']):,}")
print(f"Longueurs DLC différentes: {len(stats['dlc_counts'])}")
print(f"Niveaux de priorité utilisés: {len(stats['priority_counts'])}")

Chargement des résultats depuis le cache...

STATISTIQUES GLOBALES
Messages analysés: 5,000,000
PGNs uniques trouvés: 55
CAN IDs uniques: 68
Longueurs DLC différentes: 3
Niveaux de priorité utilisés: 6


---
## 5. Analyse du PGN 60416

### But
Calculer la proportion spécifique du PGN 60416 dans les messages.

### Contexte
Le PGN 60416 (0xEC00) est un PGN important dans le protocole J1939. Cette analyse permet de:
- Vérifier sa fréquence d'apparition
- Comprendre son importance relative dans le trafic CAN

### Interprétation
- **Fréquence élevée (>5%)**: Signal très actif, probablement critique
- **Fréquence moyenne (1-5%)**: Signal régulier
- **Fréquence faible (<1%)**: Signal occasionnel ou événementiel

In [19]:
# Analyse du PGN 60416
TARGET_PGN = 60416
pgn_60416_count = stats['pgn_counts'].get(TARGET_PGN, 0)
proportion = pgn_60416_count / stats['total_messages'] if stats['total_messages'] > 0 else 0

# Obtenir le nom du PGN
target_pgn_name = get_pgn_display_name(TARGET_PGN, PGN_NAME_MAPPING)

print(f"RÉSULTATS - PGN {TARGET_PGN} ({target_pgn_name})")
print(f"{'='*60}")
print(f"Nombre d'occurrences: {pgn_60416_count:,}")
print(f"Proportion du total: {proportion:.6%}")
print(f"Fréquence: {proportion:.8f}")

# Visualisation comparative
top_5_pgns = stats['pgn_counts'].most_common(5)
pgn_in_top5 = any(pgn == TARGET_PGN for pgn, _ in top_5_pgns)

print(f"\nPosition: ", end="")
if pgn_in_top5:
    print(f"{target_pgn_name} est dans le TOP 5 des PGNs les plus fréquents")
else:
    rank = sum(1 for _, count in stats['pgn_counts'].most_common() if count > pgn_60416_count) + 1
    print(f"Classement: #{rank} sur {len(stats['pgn_counts'])} PGNs")

RÉSULTATS - PGN 60416 (TRANSPORT_PROTOCOL_CONNECTION_MGMT)
Nombre d'occurrences: 0
Proportion du total: 0.000000%
Fréquence: 0.00000000

Position: Classement: #56 sur 55 PGNs


---
## 6. Top PGNs - Les Plus Fréquents

### But
Identifier les PGNs qui dominent le trafic CAN et analyser leur distribution.

### Pourquoi c'est important
- Les PGNs fréquents sont généralement les plus critiques pour le fonctionnement du véhicule
- La distribution de Pareto (80/20) est souvent observée: 20% des PGNs génèrent 80% du trafic
- Permet de prioriser les analyses et la détection d'anomalies

### Statistiques Calculées
- **Moyenne**: Nombre moyen d'occurrences par PGN
- **Médiane**: Valeur centrale (50% des PGNs ont moins d'occurrences)
- **Écart-type**: Mesure de la variabilité

In [20]:
# Top 20 PGNs les plus fréquents
top_pgns = stats['pgn_counts'].most_common(20)

df_pgns = pd.DataFrame(top_pgns, columns=['PGN', 'Count'])
df_pgns['Percentage'] = 100 * df_pgns['Count'] / stats['total_messages']
df_pgns['PGN_name'] = df_pgns['PGN'].apply(lambda x: get_pgn_display_name(x, PGN_NAME_MAPPING))
df_pgns['Cumulative_pct'] = df_pgns['Percentage'].cumsum()

print(f"RÉSULTATS - Top 20 PGNs les Plus Fréquents")
print(f"{'='*60}")
display(df_pgns[['PGN_name', 'PGN', 'Count', 'Percentage', 'Cumulative_pct']])

# Visualisation
fig = px.bar(df_pgns.head(15), x='PGN_name', y='Count',
             title='Top 15 PGNs par Fréquence',
             labels={'PGN_name': 'PGN', 'Count': 'Nombre de Messages'},
             color='Percentage',
             color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

# Statistiques de distribution
pgn_counts_values = list(stats['pgn_counts'].values())
print(f"\nStatistiques de Distribution des PGNs")
print(f"{'='*60}")
print(f"Moyenne d'occurrences: {np.mean(pgn_counts_values):,.2f}")
print(f"Médiane: {np.median(pgn_counts_values):,.2f}")
print(f"Écart-type: {np.std(pgn_counts_values):,.2f}")
print(f"Minimum: {np.min(pgn_counts_values):,}")
print(f"Maximum: {np.max(pgn_counts_values):,}")

# Règle 80/20
top_20pct_count = int(len(stats['pgn_counts']) * 0.2)
top_20pct_pgns = stats['pgn_counts'].most_common(top_20pct_count)
top_20pct_msgs = sum(count for _, count in top_20pct_pgns)
top_20pct_proportion = 100 * top_20pct_msgs / stats['total_messages']

print(f"\nRègle de Pareto (80/20)")
print(f"Les 20% PGNs les plus fréquents ({top_20pct_count} PGNs) représentent {top_20pct_proportion:.1f}% du trafic")

RÉSULTATS - Top 20 PGNs les Plus Fréquents


,PGN_name,PGN,Count,Percentage,Cumulative_pct
0,PROPRIETARY_B_0X80,65408,620700,12.41400,12.41400
1,ELECTRONIC_TRANSMISSION_CONTROLLER_1,61442,591148,11.82296,24.23696
2,VEHICLE_DYNAMIC_STABILITY_CONTROL_2,61449,591142,11.82284,36.05980
3,PGN_65220,65220,295578,5.91156,41.97136
4,ELECTRONIC_ENGINE_CONTROLLER_1,61444,295574,5.91148,47.88284
5,PGN_58879,58879,295574,5.91148,53.79432
6,HIGH_RESOLUTION_WHEEL_SPEED,65134,295573,5.91146,59.70578
7,PGN_65116,65116,295562,5.91124,65.61702
8,ELECTRONIC_ENGINE_CONTROLLER_2,61443,118228,2.36456,67.98158
9,PROPRIETARY_B1_0X58,130904,118228,2.36456,70.34614



Statistiques de Distribution des PGNs
Moyenne d'occurrences: 90,909.09
Médiane: 59,111.00
Écart-type: 146,945.82
Minimum: 1,178
Maximum: 620,700

Règle de Pareto (80/20)
Les 20% PGNs les plus fréquents (11 PGNs) représentent 72.7% du trafic


### Interprétation des Résultats

**Si le top 20% représente >80% du trafic**:
- Distribution très concentrée
- Quelques PGNs dominent le trafic
- Focaliser la détection d'anomalies sur ces PGNs critiques

**Si plus équilibré (top 20% ≈ 50-60%)**:
- Trafic plus diversifié
- Many signaux actifs simultanément
- Approche plus globale nécessaire

---
## 7. Analyse DLC (Data Length Code)

### But
Analyser la distribution des longueurs de données (DLC) dans les messages CAN.

### Contexte
- **DLC**: Nombre de bytes de données dans un message (0-8 pour CAN classique)
- **DLC = 0**: Messages sans données (peuvent être des heartbeats ou des erreurs)

### Pourquoi c'est important
- Messages avec DLC=0 peuvent indiquer:
  - Des messages de synchronisation
  - Des erreurs de transmission
  - Des heartbeats (signaux de vie)
- La distribution normale aide à détecter des anomalies

In [21]:
# Messages avec DLC = 0
dlc_0_count = stats['dlc_counts'].get(0, 0)

print(f"RÉSULTATS - Messages avec DLC = 0")
print(f"{'='*60}")
print(f"Nombre de messages avec DLC=0: {dlc_0_count:,}")
print(f"Proportion du total: {100 * dlc_0_count / stats['total_messages']:.4f}%")

# Distribution complète des DLC
df_dlc = pd.DataFrame([
    {'DLC': dlc, 'Count': count, 'Percentage': 100 * count / stats['total_messages']}
    for dlc, count in sorted(stats['dlc_counts'].items())
])

print(f"\nDistribution Complète des DLC")
print(f"{'='*60}")
display(df_dlc)

# DLC le plus fréquent
most_common_dlc = max(stats['dlc_counts'].items(), key=lambda x: x[1])
print(f"\nDLC le plus fréquent: {most_common_dlc[0]} bytes ({100*most_common_dlc[1]/stats['total_messages']:.2f}%)")

# Visualisation
fig = px.bar(df_dlc, x='DLC', y='Count',
             title='Distribution des Data Length Code (DLC)',
             labels={'DLC': 'Longueur des Données (bytes)', 'Count': 'Nombre de Messages'},
             text='Percentage',
             color='Count',
             color_continuous_scale='Blues')
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(height=500)
fig.show()

RÉSULTATS - Messages avec DLC = 0
Nombre de messages avec DLC=0: 0
Proportion du total: 0.0000%

Distribution Complète des DLC


,DLC,Count,Percentage
0,1,29554,0.59108
1,4,59547,1.19094
2,8,4910899,98.21798



DLC le plus fréquent: 8 bytes (98.22%)


### Interprétation

**DLC = 8 bytes dominant (>80%)**:
- Normal pour CAN classique (capacité maximale)
- La plupart des PGNs utilisent toute la capacité disponible

---
## 8. Analyse des Priorités de Messages

### But
Comprendre la distribution des niveaux de priorité utilisés dans les messages CAN.

### Contexte - Priorités CAN
Dans le protocole CAN, chaque message a une priorité (3 bits = 8 niveaux):
- **0**: Priorité la PLUS haute (messages critiques)
- **7**: Priorité la PLUS basse (messages non-critiques)

### Pourquoi c'est important
- Messages de priorité 0-2: Généralement critiques pour la sécurité (freins, direction)
- Messages de priorité 3-5: Fonctionnement normal
- Messages de priorité 6-7: Diagnostics, confort

### Pour la Détection d'Anomalies
- Anomalies sur messages haute priorité = plus critique
- Changement de priorité = potentiellement suspect

In [22]:
# Distribution des priorités
df_priority = pd.DataFrame([
    {'Priority': p, 'Count': count, 'Percentage': 100 * count / stats['total_messages']}
    for p, count in sorted(stats['priority_counts'].items())
])

print(f"RÉSULTATS - Distribution des Priorités CAN")
print(f"{'='*60}")
print("(0 = Priorité MAXIMALE, 7 = Priorité MINIMALE)\n")
display(df_priority)

# Priorité la plus utilisée
top_priority = max(stats['priority_counts'].items(), key=lambda x: x[1])
print(f"\nPriorité la plus utilisée: {top_priority[0]} ({100*top_priority[1]/stats['total_messages']:.2f}%)")

# Catégorisation
high_priority = sum(count for p, count in stats['priority_counts'].items() if p <= 2)
medium_priority = sum(count for p, count in stats['priority_counts'].items() if 3 <= p <= 5)
low_priority = sum(count for p, count in stats['priority_counts'].items() if p >= 6)

print(f"\nCatégorisation")
print(f"Haute priorité (0-2): {100*high_priority/stats['total_messages']:.2f}%")
print(f"Moyenne priorité (3-5): {100*medium_priority/stats['total_messages']:.2f}%")
print(f"Basse priorité (6-7): {100*low_priority/stats['total_messages']:.2f}%")

# Visualisation
fig = px.bar(df_priority, x='Priority', y='Count',
             title='Distribution des Niveaux de Priorité CAN',
             labels={'Priority': 'Niveau de Priorité (0=Max, 7=Min)', 'Count': 'Nombre de Messages'},
             color='Priority',
             color_continuous_scale='RdYlGn_r',
             text='Percentage')
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(height=500)
fig.show()

RÉSULTATS - Distribution des Priorités CAN
(0 = Priorité MAXIMALE, 7 = Priorité MINIMALE)



,Priority,Count,Percentage
0,2,295573,5.91146
1,3,1182292,23.64584
2,4,720165,14.40330
3,5,354682,7.09364
4,6,2349916,46.99832
5,7,97372,1.94744



Priorité la plus utilisée: 6 (47.00%)

Catégorisation
Haute priorité (0-2): 5.91%
Moyenne priorité (3-5): 45.14%
Basse priorité (6-7): 48.95%


### Interprétation

**Majorité en haute priorité (0-2)**:
- Système orienté sécurité
- Beaucoup de messages critiques
- Nécessite une détection d'anomalies très fiable

**Distribution équilibrée**:
- Mix de messages critiques et non-critiques
- Système complet avec diagnostics

**Priorité unique dominante**:
- Peut indiquer une configuration par défaut
- Ou un type de véhicule spécifique

---
## 9. Top CAN IDs - Les Plus Fréquents

### But
Identifier les CAN IDs individuels les plus actifs.

### Contexte
Un CAN ID contient:
- La priorité (3 bits)
- Le PGN (18 bits pour J1939)
- L'adresse source (8 bits)

### Utilité
- Identifier les ECUs (Electronic Control Units) les plus bavards
- Détecter les messages périodiques importants

In [23]:
# Top 20 CAN IDs
top_can_ids = stats['can_id_counts'].most_common(20)

df_can_ids = pd.DataFrame(top_can_ids, columns=['CAN_ID', 'Count'])
df_can_ids['Percentage'] = 100 * df_can_ids['Count'] / stats['total_messages']
df_can_ids['CAN_ID_hex'] = df_can_ids['CAN_ID'].apply(lambda x: f"0x{x:08X}")

print(f"RÉSULTATS - Top 20 CAN IDs les Plus Fréquents")
print(f"{'='*60}")
display(df_can_ids[['CAN_ID_hex', 'CAN_ID', 'Count', 'Percentage']])

# Visualisation
fig = px.bar(df_can_ids.head(15), x='CAN_ID_hex', y='Count',
             title='Top 15 CAN IDs par Fréquence',
             labels={'CAN_ID_hex': 'CAN ID (Hexadécimal)', 'Count': 'Nombre de Messages'},
             color='Percentage',
             color_continuous_scale='Plasma')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

RÉSULTATS - Top 20 CAN IDs les Plus Fréquents


,CAN_ID_hex,CAN_ID,Count,Percentage
0,0x0CF002E6,217055974,591148,11.82296
1,0x10FF80E6,285180134,591146,11.82292
2,0x18F009E6,418384358,591142,11.82284
3,0x0CF004E6,217056486,295574,5.91148
4,0x08FE6EE6,150892262,295573,5.91146
5,0x0CF003E6,217056230,118228,2.36456
6,0x15FF58E6,369055974,118228,2.36456
7,0x0CFE6CE6,218000614,118227,2.36454
8,0x15FF55E6,369055206,118226,2.36452
9,0x10FE6FE8,285110248,64510,1.29020


---
## 10. Corrélation PGN-Priorité

### But
Analyser si chaque PGN utilise toujours la même priorité (normal) ou si certains PGNs changent de priorité (suspect).

### Hypothèse Normale
Dans un système CAN bien configuré:
- Chaque PGN devrait avoir UNE priorité fixe
- Un PGN avec plusieurs priorités peut indiquer:
  - Une erreur de configuration
  - Une anomalie
  - Plusieurs ECUs envoyant le même PGN

### Pour CANlock
Cette analyse aide à établir le baseline "normal" pour détecter des déviations suspectes.

In [24]:
def analyze_pgn_priority_correlation(stats, top_n=10, sample_size=100000):
    """
    Analyse la relation entre PGN et priorité.
    Utilise un échantillon pour accélérer.
    """
    # Obtenir les top PGNs
    top_pgn_list = [pgn for pgn, _ in stats['pgn_counts'].most_common(top_n)]
    
    # Reparser un échantillon pour avoir la corrélation PGN-Priorité
    pgn_priority_data = defaultdict(lambda: defaultdict(int))
    
    offset = 0
    total_sampled = 0
    
    with get_session() as session:
        while total_sampled < sample_size:
            q = select(CanMessage).offset(offset).limit(BATCH_SIZE)
            batch = session.exec(q).all()
            
            if not batch:
                break
            
            for msg in batch:
                if msg.can_identifier is None:
                    continue
                
                try:
                    pgn = SessionDecoder.extract_pgn_number_from_payload(msg.can_identifier)
                    if pgn in top_pgn_list:
                        priority = (msg.can_identifier >> 26) & 0x7
                        pgn_priority_data[pgn][priority] += 1
                        total_sampled += 1
                except Exception:
                    pass
            
            offset += BATCH_SIZE
            
            if total_sampled >= sample_size:
                break
    
    # Créer DataFrame
    data_rows = []
    for pgn in top_pgn_list:
        pgn_name = get_pgn_display_name(pgn, PGN_NAME_MAPPING)
        for priority in range(8):
            count = pgn_priority_data[pgn][priority]
            if count > 0:
                data_rows.append({
                    'PGN_name': pgn_name,
                    'PGN': pgn,
                    'Priority': priority,
                    'Count': count
                })
    
    return pd.DataFrame(data_rows)

print("Analyse de la corrélation PGN-Priorité...")
print("(Échantillon de 100k messages pour accélérer)\n")

df_pgn_priority = analyze_pgn_priority_correlation(stats, top_n=10, sample_size=100000)

if not df_pgn_priority.empty:
    # Analyse: PGNs avec une seule priorité vs plusieurs
    pgn_priority_counts = df_pgn_priority.groupby('PGN_name')['Priority'].nunique()
    single_priority_pgns = sum(pgn_priority_counts == 1)
    multi_priority_pgns = sum(pgn_priority_counts > 1)
    
    print(f"RÉSULTATS - Corrélation PGN-Priorité")
    print(f"{'='*60}")
    print(f"PGNs avec une seule priorité: {single_priority_pgns}/10")
    print(f"PGNs avec plusieurs priorités: {multi_priority_pgns}/10")
    
    if multi_priority_pgns > 0:
        print(f"\nPGNs avec priorités multiples (à investiguer):")
        for pgn in pgn_priority_counts[pgn_priority_counts > 1].index:
            print(f"  - {pgn}")
    
    # Histogramme groupé par PGN
    print("\nCréation de l'histogramme...")
    
    fig = px.bar(df_pgn_priority, 
                 x='Priority', 
                 y='Count',
                 color='PGN_name',
                 barmode='group',
                 title='Distribution des Priorités par PGN (Top 10 PGNs)',
                 labels={'Priority': 'Niveau de Priorité', 'Count': 'Nombre de Messages', 'PGN_name': 'PGN'},
                 height=600)
    fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))
    fig.show()
    
    print("\nObservation: Dans un système normal, chaque PGN devrait avoir une priorité fixe (une seule barre par PGN).")
else:
    print("Pas assez de données pour l'analyse de corrélation")

Analyse de la corrélation PGN-Priorité...
(Échantillon de 100k messages pour accélérer)

RÉSULTATS - Corrélation PGN-Priorité
PGNs avec une seule priorité: 9/10
PGNs avec plusieurs priorités: 1/10

PGNs avec priorités multiples (à investiguer):
  - PROPRIETARY_B_0X80

Création de l'histogramme...



Observation: Dans un système normal, chaque PGN devrait avoir une priorité fixe (une seule barre par PGN).


### Interprétation

**Tous les PGNs ont une seule priorité** (10/10):
- Configuration normale et cohérente
- Le système est bien configuré
- Baseline clair pour la détection d'anomalies
- Sur l'histogramme: chaque PGN n'apparaît que sur une seule valeur de priorité
